In [1]:
import pandas as pd
import datetime

In [2]:
date_str = datetime.datetime.now().strftime("%Y-%m-%d")

#date_str = '2018-05-31'

print("Date: " + date_str)

Date: 2018-06-18


In [3]:
# Read the Calendar extract to get additional details
cal_1 = pd.read_json('/Users/taj/GitHub/scraping/stayz/WebData/nsw_calendar/stayz_calendar_' + date_str + '.json')



cal_2 = cal_1[cal_1['property_id'].notnull()]

cal_2.set_index('property_id',inplace=True)


cal = cal_2

cal.head()

,calendar,ext_at,features,photos,review_count,review_value
property_id,,,,,,
9174814,"<div class=""ui-datepicker-group ui-datepicker-...",2018-06-18 18:20:37,Air conditioning\nChild's high chair\nChildren...,20,8,4.8
9155220,"<div class=""ui-datepicker-group ui-datepicker-...",2018-06-18 18:20:40,Air conditioning\nBooks\nChild's high chair\nC...,20,37,5.0
9170531,"<div class=""ui-datepicker-group ui-datepicker-...",2018-06-18 18:20:43,Air conditioning\nBalcony\nBooks\nDishes and u...,8,8,4.8
9193783,"<div class=""ui-datepicker-group ui-datepicker-...",2018-06-18 18:20:46,Children welcome\nDining\nDishes and utensils\...,8,6,4.7
9210143,"<div class=""ui-datepicker-group ui-datepicker-...",2018-06-18 18:20:49,Air conditioning\nBalcony\nClothes dryer\nDeck...,12,9,5.0


In [4]:
position_dict = {}
position_index = 0


data_dict = {}


# Step one - Create the dictionary of all possible feature items
for index, row in cal.iterrows():
    
    a = row['features']
    
    #print("Features: " + a)
    #save_features = [0] * len(position_dict)
    
    b = a.split('\n')
    
    for b1 in b:
        # Check if it exists in the dictionary
        
        if ( len(b1) > 0 ):
            if b1 in position_dict:
                b1_pos = position_dict[b1]
                #save_features[b1_pos] = 1

            else:
                position_dict[b1] = position_index

                b1_pos = position_index

                position_index += 1
            
print(position_dict)



{'Air conditioning': 0, "Child's high chair": 1, 'Children welcome': 2, 'Clothes dryer': 3, 'Coffee maker': 4, 'Dining': 5, 'Dishes and utensils': 6, 'DVD player': 7, 'Fireplace': 8, 'Heating': 9, 'Iron and board': 10, 'Kitchen': 11, 'Linens provided': 12, 'Microwave': 13, 'Non-Smoking': 14, 'Outdoor grill': 15, 'Oven': 16, 'Parking': 17, 'Refrigerator': 18, 'Stove': 19, 'Television': 20, 'Toaster': 21, 'Towels provided': 22, 'Washing machine': 23, 'Books': 24, 'Deck/Patio': 25, 'Dishwasher': 26, 'Games': 27, 'Games room': 28, 'Hair dryer': 29, 'Internet access': 30, 'Lawn/Garden': 31, 'Living room': 32, 'Stereo': 33, 'Video library': 34, 'Wheelchair accessible': 35, 'Balcony': 36, 'Long-term renters welcome': 37, 'Pets allowed': 38, 'Garage': 39, 'Wood stove': 40, 'Satellite/Cable': 41, 'Toys': 42, 'Communal pool': 43, 'Hot tub': 44, 'Events allowed': 45, 'Telephone': 46, 'Fitness room/Equipment': 47, 'Video Games': 48, 'Indoor pool': 49, 'Heated pool': 50, 'Private pool': 51, 'Tennis

In [5]:

# Step 2 - scan and create the data
for index, row in cal.iterrows():
    #print(index)
    a = row['features']
    b = a.split('\n')
    
    save_features = [0] * len(position_dict)
    
    for b1 in b:    
        if ( len(b1) > 0 ):
            if b1 in position_dict:
                b1_pos = position_dict[b1]
                save_features[b1_pos] = 1
            else:
                print("ERROR: found a new feature not in dictionary: '" + b1 + "'")
    #print("Index: " + index)
    #if ( index > 0 ):
    data_dict[index] = save_features
    
df1 = pd.DataFrame.from_dict(data_dict)


df2 = df1.T

col_n_1 = position_dict.keys()

col_n_2 = []


for c in col_n_1:
    # Cleaning text
    for char in '\'/-.,\n ':
        c=c.replace(char,'_')

    # All words lower case
    c = c.lower()
    
    col_n_2.append('f_' + c)

df2.columns = col_n_2

df2.head()

,f_air_conditioning,f_child_s_high_chair,f_children_welcome,f_clothes_dryer,f_coffee_maker,f_dining,f_dishes_and_utensils,f_dvd_player,f_fireplace,f_heating,...,f_communal_pool,f_hot_tub,f_events_allowed,f_telephone,f_fitness_room_equipment,f_video_games,f_indoor_pool,f_heated_pool,f_private_pool,f_tennis
4840533,0,0,0,0,0,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4841630,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4845880,1,1,1,1,1,1,1,0,0,1,...,0,0,1,0,0,0,0,0,0,0
4848261,1,0,0,1,1,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4887073,1,0,0,0,0,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# Write out the file to CSV so it can be joined in the main branch

csv_out = df2.to_csv()

fh = open('/Users/taj/GitHub/scraping/stayz/WebData/nsw_features/stayz_nsw_features_' + date_str + '.csv', 'w')

fh.write(csv_out)

fh.close()